# 1. Exploration des fichiers csv

In [9]:
import pandas as pd
import pickle
import os
import numpy as np

# 1. Chargement des métadonnées
metadata = pd.read_csv("P10/articles_metadata.csv")
print("📰 Articles metadata")
print(metadata.head())
print(metadata.info())
print(metadata.isnull().sum())

# 2. Chargement des interactions (échantillon)
clicks_sample = pd.read_csv("P10/clicks_sample.csv")
print("\n👆 Clicks sample")
print(clicks_sample.head())
print(clicks_sample.info())
print(f"{clicks_sample['user_id'].nunique()} utilisateurs uniques")
print(f"{clicks_sample['click_article_id'].nunique()} articles différents")

# 3. Chargement des embeddings
with open("P10/articles_embeddings.pickle", "rb") as f:
    embeddings = pickle.load(f)  # shape = (364047, 250)

# Création d’un DataFrame avec les mêmes index que metadata
article_embeddings_df = pd.DataFrame(
    embeddings,
    index=metadata['article_id'],
    columns=[f'embedding_{i}' for i in range(embeddings.shape[1])]
)

print("\n🔢 Embeddings (vecteurs d'articles)")
print(article_embeddings_df.head())
print("Index des embeddings (DataFrame):", list(article_embeddings_df.index[:5]))
print("IDs articles metadata :", list(metadata['article_id'].head()))

# 4. Vérification des correspondances avec les articles cliqués
article_ids_embeddings = set(article_embeddings_df.index)
clicked_articles = set(clicks_sample['click_article_id'])

articles_absents = clicked_articles - article_ids_embeddings
print(f"\n⚠️ Articles cliqués sans vecteur d'embedding : {len(articles_absents)}")
if articles_absents:
    print("Exemples :", list(articles_absents)[:5])


📰 Articles metadata
   article_id  category_id  created_at_ts  publisher_id  words_count
0           0            0  1513144419000             0          168
1           1            1  1405341936000             0          189
2           2            1  1408667706000             0          250
3           3            1  1408468313000             0          230
4           4            1  1407071171000             0          162
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 364047 entries, 0 to 364046
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype
---  ------         --------------   -----
 0   article_id     364047 non-null  int64
 1   category_id    364047 non-null  int64
 2   created_at_ts  364047 non-null  int64
 3   publisher_id   364047 non-null  int64
 4   words_count    364047 non-null  int64
dtypes: int64(5)
memory usage: 13.9 MB
None
article_id       0
category_id      0
created_at_ts    0
publisher_id     0
words_count      0
dtype: int64

👆 

- 364 047 articles complets
- 250 dimensions, aucun article manquant
- 707 utilisateurs, 323 articles, interactions propres
- Correspondance parfaite (aucun manquant) articles cliqués ↔ embeddings

# 2. Exploration des fichiers clicks_hours

In [10]:
# Lister les fichiers

clicks_dir = "P10/clicks"
clicks_files = sorted([os.path.join(clicks_dir, f) for f in os.listdir(clicks_dir) if f.endswith(".csv")])

print(f"🗂️ {len(clicks_files)} fichiers trouvés")
print("Exemples :", clicks_files[:5])

🗂️ 385 fichiers trouvés
Exemples : ['P10/clicks/clicks_hour_000.csv', 'P10/clicks/clicks_hour_001.csv', 'P10/clicks/clicks_hour_002.csv', 'P10/clicks/clicks_hour_003.csv', 'P10/clicks/clicks_hour_004.csv']


In [11]:
# Charger les 3 premiers fichiers

sample_dfs = [pd.read_csv(f) for f in clicks_files[:3]]
clicks_large_sample = pd.concat(sample_dfs, ignore_index=True)

print("📊 Aperçu d'un échantillon élargi")
print(clicks_large_sample.head())
print(clicks_large_sample.info())
print(clicks_large_sample['user_id'].nunique(), "utilisateurs uniques")
print(clicks_large_sample['click_article_id'].nunique(), "articles différents")

📊 Aperçu d'un échantillon élargi
   user_id        session_id  session_start  session_size  click_article_id  \
0        0  1506825423271737  1506825423000             2            157541   
1        0  1506825423271737  1506825423000             2             68866   
2        1  1506825426267738  1506825426000             2            235840   
3        1  1506825426267738  1506825426000             2             96663   
4        2  1506825435299739  1506825435000             2            119592   

   click_timestamp  click_environment  click_deviceGroup  click_os  \
0    1506826828020                  4                  3        20   
1    1506826858020                  4                  3        20   
2    1506827017951                  4                  1        17   
3    1506827047951                  4                  1        17   
4    1506827090575                  4                  1        17   

   click_country  click_region  click_referrer_type  
0              1 

In [14]:
# Vérification de la structure

expected_cols = set(clicks_sample.columns)
for i, df in enumerate(sample_dfs):
    if set(df.columns) != expected_cols:
        print(f"⚠️ Structure différente dans {clicks_files[i]}")

In [13]:
# Fusion complète 

all_clicks_df = pd.concat([pd.read_csv(f) for f in clicks_files], ignore_index=True)
print("✅ Fichiers fusionnés :", all_clicks_df.shape)

✅ Fichiers fusionnés : (2988181, 12)


- Fichiers analysés	--> ~300 fichiers
- Données fusionnées --> 2 988 181 lignes, 12 colonnes
- Utilisateurs uniques --> ≈ 1,5 million
- Articles cliqués	--> plusieurs milliers (≥ 500)
- Structure	Identique à clicks_sample.csv 

# 3. Test

In [17]:
import sys
sys.path.append("P10")
from recommend_articles import Recommender

# Création de l'objet recommender
reco = Recommender(clicks_sample, article_embeddings_df)

# Exemple pour un utilisateur connu
user_id = clicks_sample['user_id'].iloc[0]
recommended_articles = reco.recommend(user_id)

print(f"Top 5 recommandations pour l’utilisateur {user_id} : {recommended_articles}")


Top 5 recommandations pour l’utilisateur 0 : [157519, 162856, 159495, 157944, 156690]


In [20]:
import pandas as pd
from IPython.display import display

# 🔁 Étape 1 : Générer les recommandations pour un échantillon d'utilisateurs
test_users = clicks_sample['user_id'].unique()[:20]  # Ajustable

recommendation_results = []

for uid in test_users:
    articles = reco.recommend(uid)
    for rank, article_id in enumerate(articles):
        recommendation_results.append({
            'user_id': uid,
            'rank': rank + 1,
            'recommended_article_id': article_id
        })

# 📄 Étape 2 : Création du DataFrame
recommendations_df = pd.DataFrame(recommendation_results)

# 🔗 Étape 3 : Enrichissement avec les métadonnées
reco_with_meta = recommendations_df.merge(
    metadata,
    left_on='recommended_article_id',
    right_on='article_id',
    how='left'
)

# 🕓 Étape 4 : Conversion du timestamp en date lisible
reco_with_meta['created_at'] = pd.to_datetime(reco_with_meta['created_at_ts'], unit='ms')

# 🎯 Étape 5 : Sélection des colonnes à afficher
final_cols = [
    'user_id',
    'rank',
    'recommended_article_id',
    'category_id',
    'words_count',
    'created_at'
]

# 📊 Étape 6 : Affichage propre, trié par utilisateur et rang
display(
    reco_with_meta[final_cols]
    .sort_values(by=["user_id", "rank"])
    .reset_index(drop=True)
    .head(20)  # Affiche les 20 premières recommandations
)


,user_id,rank,recommended_article_id,category_id,words_count,created_at
0,0,1,157519,281,272,2017-09-22 07:22:32
1,0,2,162856,281,237,2017-11-18 10:24:44
2,0,3,159495,281,261,2017-09-21 19:19:21
3,0,4,157944,281,204,2017-12-05 20:11:07
4,0,5,156690,281,170,2017-06-07 18:19:39
5,1,1,95797,209,195,2017-05-24 20:00:01
6,1,2,90583,199,158,2017-12-27 19:16:51
7,1,3,246739,386,128,2017-12-25 14:30:20
8,1,4,231234,375,139,2015-11-10 15:17:30
9,1,5,90167,199,157,2018-02-27 13:12:50
